<a href="https://colab.research.google.com/github/Hemapriya-aids/machine-learining-internship/blob/main/task4_machine_learning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [12]:
# ===============================================================
# CODTECH INTERNSHIP - TASK 4  (No Surprise Library)
# Simple Collaborative Filtering using Pandas & Numpy
# ===============================================================

import pandas as pd
import numpy as np

# Step 1: Create dataset
data = {
    'user_id': [1,1,1,2,2,3,3,4,4,5],
    'movie'  : ['Avengers','Iron Man','Thor',
                'Iron Man','Captain America',
                'Avengers','Thor',
                'Iron Man','Captain America',
                'Avengers'],
    'rating' : [5,4,4,5,3,4,5,4,3,5]
}
df = pd.DataFrame(data)

# Step 2: Build user-item matrix
user_movie_matrix = df.pivot_table(index='user_id', columns='movie', values='rating')
print("🎬 User-Movie Matrix:\n", user_movie_matrix)

# Step 3: Compute movie-to-movie similarity (cosine)
from numpy.linalg import norm

def cosine_sim(a,b):
    mask = ~np.isnan(a) & ~np.isnan(b)
    if np.sum(mask)==0: return 0
    a,b = a[mask], b[mask]
    return np.dot(a,b) / (norm(a)*norm(b))

movies = user_movie_matrix.columns
similarity = pd.DataFrame(index=movies, columns=movies)

for m1 in movies:
    for m2 in movies:
        similarity.loc[m1,m2] = cosine_sim(
            user_movie_matrix[m1].values,
            user_movie_matrix[m2].values
        )

print("\n🎞️ Movie Similarity Matrix:\n", similarity)

# Step 4: Simple recommendation function
def recommend_movies(user_id, top_n=3):
    user_ratings = user_movie_matrix.loc[user_id]
    unseen = user_ratings[user_ratings.isna()].index
    scores = {}
    for movie in unseen:
        sim_scores = []
        for rated_movie, rating in user_ratings.dropna().items():
            sim = float(similarity.loc[movie, rated_movie])
            sim_scores.append(sim * rating)
        if sim_scores:
            scores[movie] = np.sum(sim_scores) / np.sum(np.abs([float(similarity.loc[movie, r]) for r in user_ratings.dropna().index]))
    ranked = sorted(scores.items(), key=lambda x:x[1], reverse=True)
    return ranked[:top_n]

# Step 5: Show recommendations
user_id = 2
print(f"\n⭐ Recommendations for User {user_id}:")
for movie, score in recommend_movies(user_id):
    print(f"{movie}  (Predicted score: {score:.2f})")


🎬 User-Movie Matrix:
 movie    Avengers  Captain America  Iron Man  Thor
user_id                                           
1             5.0              NaN       4.0   4.0
2             NaN              3.0       5.0   NaN
3             4.0              NaN       NaN   5.0
4             NaN              3.0       4.0   NaN
5             5.0              NaN       NaN   NaN

🎞️ Movie Similarity Matrix:
 movie           Avengers Captain America  Iron Man     Thor
movie                                                      
Avengers             1.0               0       1.0  0.97561
Captain America        0             1.0  0.993884        0
Iron Man             1.0        0.993884       1.0      1.0
Thor             0.97561               0       1.0      1.0

⭐ Recommendations for User 2:
Avengers  (Predicted score: 5.00)
Thor  (Predicted score: 5.00)
